In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import cv2

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 10s 2us/step


In [4]:
def LeNet5(input_shape=(28, 28, 1), num_classes=10):
    model = tf.keras.Sequential([
        layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [5]:
def rcnn_model(input_shape=(28, 28, 1)):
    inputs = layers.Input(input_shape)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(256, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    scores = layers.Dense(1, activation='sigmoid')(x)
    bbox = layers.Dense(4)(x)
    class_probs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=[scores, bbox, class_probs])
    return model

In [6]:
rcnn = rcnn_model(input_shape=(224, 224, 1))
lenet5 = LeNet5(input_shape=(28, 28, 1))

In [7]:
rcnn.compile(loss=['binary_crossentropy', 'mse', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
lenet5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
x_train = np.resize(x_train, (60000, 224, 224, 1))
x_test = np.resize(x_test, (10000, 224, 224, 1))
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [10]:
rcnn.fit(x_train, [np.ones(60000), np.zeros((60000, 4)), y_train], batch_size=32, epochs=1, validation_data=(x_test, [np.ones(10000), np.zeros((10000, 4)), y_test]))

1875/1875 [==============================] - 6150s 3s/step - loss: 2.3207 - dense_2_loss: 0.0054 - dense_3_loss: 0.0056 - dense_4_loss: 2.3097 - dense_2_accuracy: 1.0000 - dense_3_accuracy: 0.2751 - dense_4_accuracy: 0.1001 - val_loss: 2.3043 - val_dense_2_loss: 7.6786e-04 - val_dense_3_loss: 4.2877e-06 - val_dense_4_loss: 2.3036 - val_dense_2_accuracy: 1.0000 - val_dense_3_accuracy: 1.0000 - val_dense_4_accuracy: 0.1000


In [19]:
x_test1 = x_test[:100]
y_test1 = y_test[:100]
print(x_test1[0])

[[[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.14509805]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.6117647 ]
  [0.69803923]
  [0.        ]]

 [[0.01176471]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 ...

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.49803922]
  [0.23137255]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.08235294]
  [0.07058824]
  [0.14509805]]

 [[0.        ]
  [0.03137255]
  [0.01960784]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]]


In [20]:
class_probs = rcnn.predict(x_test1)

4/4 [==============================] - 2s 465ms/step


In [22]:
rcnn_output = rcnn.predict(x_test[:100])
bbox_preds, class_probs = rcnn_output[0], rcnn_output[1]

4/4 [==============================] - 2s 454ms/step


In [34]:
def generate_bboxes(bbox_preds):
    bboxes = []
    for i in range(len(bbox_preds)):
        x1 = int(bbox_preds[i][0] * 28)
        y1 = int(bbox_preds[i][0][0] * 28)
        x2 = int(bbox_preds[i][2] * 28)
        y2 = int(bbox_preds[i][3] * 28)
        bboxes.append((x1, y1, x2, y2))
    return bboxes

In [29]:
def classify_clothes(x_test, bboxes, class_probs, lenet5):
    pred_classes = []
    for i in range(len(bboxes)):
        x1, y1, x2, y2 = bboxes[i]
        img = cv2.resize(x_test[i, y1:y2, x1:x2, :], (28, 28), interpolation=cv2.INTER_AREA)
        img = img.reshape((1, 28, 28, 1))
        pred_class = lenet5.predict(img)
        pred_classes.append(pred_class)
    return pred_classes

In [33]:
total_correct = 0
for i in range(len(pred_classes)):
    if np.argmax(pred_classes[i]) == np.argmax(y_test[i]):
        total_correct += 1
accuracy = total_correct / len(pred_classes)
print('Overall accuracy:', accuracy)

NameError: name 'pred_classes' is not defined

In [35]:
bboxes = generate_bboxes(bbox_preds)
pred_classes = classify_clothes(x_test1, bboxes, class_probs, lenet5)

IndexError: invalid index to scalar variable.